This imports the packages we need to create the interactive map. I had to make a separate conda environment for geomapping using geopandas.

In [2]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

import folium as folium

import bokeh
from bokeh.plotting import figure, save

from newick import loads
from newick import read

from Bio import Phylo

import panel as pn
import param as pr

In [3]:
#conda list

## Datasets

* I downloaded the sequences from [ViruSurf](http://geco.deib.polimi.it/virusurf/) and chose regions found in Gujarat. This made a FASTA file of 398 full nucleotide sequences. 

* Each of the viral strains in this dataset is collected from the Gujarat area of India. I limited the set to 398 due to computation limitations though there were more

* Most of the strains (179) were collected from the city of Ahmedabad in Gujarat followed by Vadodara and Surat

* Some of the regions from which the viral strains have been collected are municipalities instead of cities. I chose representative cities to mark these viral strains on the map. 

### Whole Phylogeny Tree

This phylogeny tree can help us visualize the different strains chosen

In [4]:
tree = next(Phylo.parse("RAxML_bipartitionsBranchLabels.RAXML.gujarat_viral_fin", "newick"))
Phylo.draw_ascii(tree)

   ______ MT799991.1
 ,|
 ||    __ MT676013.1
 ||___|
 |    |_ MT676014.1
 |
 |   _ MW173252.1
 | _|
 || |___ MW173253.1
 ||
 || ______ MT451887.1
 |||
 |||                   _________________ MT451890.1
 |||              ____|
 |||             |    | MT451889.1
 |||       ______|
 |||      |      |     _ MT451878.1
 |||      |      |  __|
 |||      |      |_|  |_ MT451880.1
 |||      |        |
 ||| _____|        |___________ MT451874.1
 ||||     |
 ||||     |  _______________________ MT759588.1
 ||||     | |
 ||||     | |     _____ MT451875.1
 ||||     |_|    |
 ||||       |    |  ______ MT635409.1
 ||||       |    | |
 ||||       |____| |______ MT759714.1
 ||||            | |
 ||||            | |________ MT451877.1
 ||||            | |
 ||||            |_|_____ MT451879.1
 ||||              |
 ||||              |____ MT800010.1
 ||||              |
 ||||              |____ MT740923.1
 ||||
 ||||    ________ MW242689.1
 ||||  _|
 |||| | |                    ________________________ M

```<img src="Gujarat_ViralGenomeTree_highq.png"
     alt="Gujarat Phylogenetic Tree"
     style="float: left; margin-right: 10px;" />```

In [5]:
viral = gpd.read_file("result.csv")
viral = viral.drop({"is_reference", "source_page", "strand", "length", "sequencing_technology", "database_source", "gender", "age", "assembly_method", "coverage", "bioproject_id", "isolation_source"}, axis = 1)
viral = viral.rename({"region" : "city"}, axis = 1)
viral = viral.replace(to_replace = "Gujarat, ", value = "", regex = True)

regions = viral.groupby("city").count()
regions = pd.DataFrame(regions["accession_id"])
regions = regions.rename({"accession_id" : "viral_strains"}, axis = 1).reset_index()

regions["rank"] = regions["viral_strains"].rank(method = "dense")
regions = regions.sort_values("viral_strains", ascending = True).reset_index().drop("index", axis = 1)
regions.head()

,city,viral_strains,rank
0,Una,1,1.0
1,Dhanera,1,1.0
2,Fatepura,1,1.0
3,Jamkhambhaliya,1,1.0
4,Junagadh,1,1.0


* There were some collection centers which were labelled on a municipality rather than city level. To bring it down to a city level, I had to find a representative latitude and longitude present in our cities dataset. 

* This is not the most accurate way to do this and can be improved in the future

* For Daskroi, I went through a list of the cities in this municipality and used the coordinates for one of its cities - Jetalpur

In [6]:
##Cities in Daskroi municipality

##Renaming municipality to main district
regions[regions["city"] == "Daskroi"]
regions.iloc[16, 0] = "Jetalpur"
regions.head()


,city,viral_strains,rank
0,Una,1,1.0
1,Dhanera,1,1.0
2,Fatepura,1,1.0
3,Jamkhambhaliya,1,1.0
4,Junagadh,1,1.0


* I got the indian cities database from [github user sanand0](https://github.com/sanand0/pincode/blob/master/data/IN.csv) and selected for cities that were found in the state of Gujarat.

* This is to find the coordinates of each of these cities to mark in the interactive map
* For the cities that were not represented in this dataset, I had to manually enter latitude and longitude found from Google Maps/Wikipedia

In [7]:
##Data cleaning and renaming

cities = pd.read_csv("gujarat_city.csv")
cities = cities.drop(["key", "accuracy", "admin_name1"], axis = 1)
cities = cities.rename({"place_name" : "city"}, axis = 1)

#These are the cities found in the viral strains for which we need to manually enter coordinates/match with alternate names in the cities dataset
set(regions["city"]) - set(cities["city"])


{'Bodeli',
 'Dahod',
 'Daskroi',
 'Dhanera',
 'Fatepura',
 'Gandhinagar',
 'Godhra',
 'Jamkhambhaliya',
 'Junagadh',
 'Kalyanpur',
 'Kodinar',
 'Mahemdavad',
 'Mehsana',
 'Nadiad',
 'Surat',
 'Sutrapada',
 'Una',
 'Veraval'}

In [8]:
#set(cities["city"])

In [9]:
##Renaming
cities.iloc[562, 0] = "Dahod"
cities.iloc[382, 0] = "Fatepura"
cities.iloc[247, 0] = "Gandhinagar"
cities.iloc[264, 0] = "Kalyanpur"
cities.iloc[383, 0] = "Mehasana"
cities.iloc[474, 0] = "Nadiad"

##Adding other latitudes and longitudes
cities.loc[758] = ["Surat", 21.1702, 72.8311]
cities.loc[759] = ["Sutrapada", 20.82, 71.03]
cities.loc[760] = ["Una", 20.82, 71.03]
cities.loc[761] = ["Kodinar", 20.86, 70.80]
cities.loc[762] = ["Veraval", 20.91, 70.37]
cities.loc[763] = ["Junagadh", 21.520, 70.463]
cities.loc[764] = ["Jamkhambhaliya", 22.120, 69.39]
cities.loc[765] = ["Godhra", 22.4638, 73.3713]
cities.loc[766] = ["Dhanera", 24.52, 72.02]
cities.loc[767] = ["Bodeli", 22.27914, 73.71036]
cities.loc[768] = ["Mahemdavad", 22.83, 72.77]

cities.head()


,city,latitude,longitude
0,Parsi Agiyari Chwok,22.0833,70.9167
1,Rajkot Bhaktinagar,22.0833,70.9167
2,Aji Industrial Estate,22.0833,70.9167
3,Rajkot S N Gurukul,22.0833,70.9167
4,University Campus,22.0833,70.9167


In [59]:
covid = pd.merge(left = viral, right = cities, on = "city")

covid = gpd.GeoDataFrame(covid, crs = "EPSG:4326", geometry = gpd.points_from_xy(covid.longitude, covid.latitude))
covid = covid.drop({"latitude", "longitude", "lineage_clade"}, axis = 1)
covid.head(2)

##Check to see if latitude and longitude has been filled for all collection centers
covid[covid["collection_date"] == "2020-04-07"]

covid[covid["city"] == "Una"]

,accession_id,strain_name,is_complete,gc_percentage,n_percentage,submission_date,collection_date,country,city,taxon_name,taxon_id,geometry
385,MW242780.1,SARS-CoV-2/human/IND/GBRC425/2020,true,37.99,0,2020-11-11,2020-07-14,India,Una,Severe acute respiratory syndrome coronavirus 2,2697049,POINT (71.03000 20.82000)


From here, we can see group our viral strains according to the collection date to start building the Gujarat dashboard. Looking at the table below, we can tell that the highest number of strains (37) were collected on 2020-06-15.	

In [58]:
#This was to see if there are duplicate accession IDs - this was false
#covid.duplicated("accession_id")

dates = covid.groupby("collection_date").count()
dates = dates["accession_id"]
dates = dates.reset_index()
dates.head()


,collection_date,accession_id
0,2020-04-07,1
1,2020-04-11,2
2,2020-04-13,4
3,2020-04-14,1
4,2020-04-22,1


## Making the Gujarat bubble map using Folium

This represents cities with more viral strains than others. As we can see, Ahmedabad has the highest number of strains.

In [12]:
map_initial = folium.Map(location = [22.2587, 71.1924], zoom_start = 7, tiles = "Stamen Toner")

def markSize(city):
    line = regions[regions["city"] == city]
    return int(line["rank"])

def numStrains(city):
    line = regions[regions["city"] == city]
    return int(line["viral_strains"])

for i in covid.itertuples():
    folium.CircleMarker(location = [i[12].xy[1][0], i[12].xy[0][0]], color = "cadetblue", fill = True, fill_color = "lightblue", popup = "Name: " + i[9] + "; " + "Strains: " + str(numStrains(i[9])), radius = 5 * markSize(i[9]), weight = 1).add_to(map_initial)

map_initial.save("map.html")
map_initial



## Making a collection time dashboard of the COVID-19 strains in Gujarat

The code below uses an if statement to differentiate between the strains present in Gujarat at different collection times

In [57]:
##Here, the size of the marker depends on the number of strains found in that city on that collection date. 
##ie. more number of strains on that collection date means larger radius

def writeMarkers(date = "2020-11-24"):
    map_dash = folium.Map(location = [22.2587, 71.1924], zoom_start = 7, tiles = "Stamen Toner")
    
    for i in covid.itertuples():
        if i[7] == date:
            viral = covid[covid["collection_date"] == date]
            cov_date = viral.groupby("city").count().reset_index()
            cov_date = cov_date.sort_values(by = "accession_id", ascending = False)
            cov_date["rank"] = cov_date["accession_id"].rank(method = "dense")
            per_city = cov_date[cov_date["city"] == i[9]]
            rank = int(per_city["rank"])
            num = int(per_city["accession_id"])
            folium.CircleMarker(location = [i[12].xy[1][0], i[12].xy[0][0]], color = "crimson", fill = True, fill_color = "crimson", popup = "Name: " + i[9] + "; " + "Number of Strains: " + str(num), radius = 10 * rank, weight = 1).add_to(map_dash)
    
    return map_dash

pn.extension()

kw = dict(date = sorted(list(dates["collection_date"])))
i = pn.interact(writeMarkers, **kw)
i.pprint()

text = "<b>Time Series of COVID-19 in Gujarat</b>"
p = pn.Row(i[1][0], pn.Column(text, i[0][0]))
p





Column
    [0] Column
        [0] Select(name='date', options=['2020-04-07', ...], value='2020-11-24')
    [1] Row
        [0] Folium(Map, name='interactive00402')


Row
    [0] Folium(Map, name='interactive00402')
    [1] Column
        [0] Markdown(str)
        [1] Select(name='date', options=['2020-04-07', ...], value='2020-11-24')